In [1]:
# Importació de llibreries essencials per al processament de text, aprenentatge automàtic i manipulació de dades
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# Descàrrega de recursos de NLTK per a la tokenització, lematització i etiquetatge gramatical
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

True

In [4]:
# Funció per eliminar emojis mitjançant una expressió regular que cobreix la majoria de rangs Unicode corresponents
def eliminar_emojis(text):
    if isinstance(text, str):
        patron = re.compile("[" 
            "\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF" "\U0001F700-\U0001F77F"
            "\U0001F780-\U0001F7FF" "\U0001F800-\U0001F8FF"
            "\U0001F900-\U0001F9FF" "\U0001FA00-\U0001FA6F"
            "\U0001FA70-\U0001FAFF" "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251" "]+", flags=re.UNICODE)
        return patron.sub(r'', text)
    return text

# Inicialització del lematitzador basat en WordNet
lematitzador = WordNetLemmatizer()

# Funció auxiliar per obtenir l'etiqueta gramatical (POS) d'una paraula, necessària per lematitzar correctament
def obtenir_pos_tag(paraula):
    etiqueta = nltk.pos_tag([paraula])[0][1][0].upper()
    tipus = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tipus.get(etiqueta, wordnet.NOUN)  # Per defecte, es considera un nom (noun)

# Funció principal de lematització: tokenitza el text i aplica la lematització amb POS
def lematitzar_text(text):
    paraules = nltk.word_tokenize(text)
    return ' '.join([lematitzador.lemmatize(p, obtenir_pos_tag(p)) for p in paraules])


# Carreguem el conjunt de dades SMS etiquetat (ham/spam) des d’una font pública
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=['etiqueta', 'missatge'])

# Conversió de les etiquetes textuals a valors binaris: 'ham' → 0 i 'spam' → 1
df['etiqueta'] = df['etiqueta'].replace({'ham': 0, 'spam': 1})

# Aplicació del preprocessament: eliminació d’emojis i lematització dels missatges
df['missatge'] = df['missatge'].apply(eliminar_emojis).apply(lematitzar_text)

# Definició de stopwords (paraules buides) que seran excloses del model de TF-IDF
stopwords = ['a', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
            'is', 'it', 'for', 'with', 'that', 'this', 'as', 'was', 'be',
            'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
            'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not']

In [5]:
i=0
# Assignació de variables predictives (X) i de la variable objectiu (y)
X_text = df['missatge']
y = df['etiqueta']

# Configuració de validació creuada estratificada amb 5 particions,
# mantenint la proporció de classes en cada partició
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

# Bucle principal de validació creuada
for train_idx, test_idx in skf.split(X_text, y):
    # Separació del conjunt d'entrenament i de prova
    X_train_text = X_text.iloc[train_idx]
    X_test_text = X_text.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]

    # Vectorització dels textos utilitzant TF-IDF amb unigramas i bigramas
    # i un límit de 10.000 característiques. L’ajust es fa només amb el conjunt d’entrenament.
    vectoritzador = TfidfVectorizer(max_features=10000, stop_words=stopwords)
    X_train = vectoritzador.fit_transform(X_train_text)
    X_test = vectoritzador.transform(X_test_text)

    # Inicialització i entrenament del model XGBoost amb hiperparàmetres definits
    model = XGBClassifier(
        n_estimators=500,        # Nombre d’arbres del model
        max_depth=6,             # Profunditat màxima de cada arbre
        learning_rate=0.01,      # Taxa d’aprenentatge
        random_state=42          # Semilla per a resultats reproductibles
    )
    model.fit(X_train, y_train)
    print("Iteracio: "+str(i) )  # Impressió per indicar el progrés de la iteració


    # Predicció de probabilitats per a la classe positiva (spam) i càlcul de l'AUC-ROC
    probabilitats = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, probabilitats)
    scores.append(auc)
    print("AUC-ROC: Fold " + str(i) +  " : " + str(auc))  # Impressió de la puntuació AUC del fold actual
    i=i+1

Iteracio: 0
AUC-ROC: Fold 0:0.9806632124352331
Iteracio: 1
AUC-ROC: Fold 1:0.9831778929188255
Iteracio: 2
AUC-ROC: Fold 2:0.9831206314984179
Iteracio: 3
AUC-ROC: Fold 3:0.9789825086066001
Iteracio: 4
AUC-ROC: Fold 4:0.9868484195152484


In [6]:
# Organitzem els resultats
resultats=[]
resultats.append({
        'k_components': 10000,
        'auc_roc_mitjana': np.mean(scores)
    })

df_resultats = pd.DataFrame(resultats)

# Mostrar la taula de resultats
print("\n Resultats de validació creuada (mitjana AUC-ROC per a k=10000):\n")
print(df_resultats.to_string(index=False, float_format="%.4f"))


 Resultats de validació creuada (mitjana AUC-ROC per a k=10000):

 k_components  auc_roc_mitjana
        10000           0.9826
